# Forecast de production photovoltaïque avec PVLib

In [ ]:

import mlflow.pyfunc
import pvlib
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.pvsystem import PVSystem, FixedMount, Array


class PVLibModel(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.module_parameters = {
            'pdc0': 240, 'gamma_pdc': -0.004
        }
        self.temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm'][
            'open_rack_glass_glass']
        self.inverter_parameters = {
            'pdc0': 240, 'eta_inv_nom': 0.96
        }

    def load_context(self, context):
        pass

    def predict(self, model_input):
        row = model_input.iloc[0]
        location = Location(
            latitude=row['latitude'],
            longitude=row['longitude'],
            name=row['name'],
            altitude=row['altitude'],
            tz=row['timezone']
        )
        mount = FixedMount(surface_tilt=row['latitude'], surface_azimuth=180)
        array = Array(
            mount=mount,
            module_parameters=self.module_parameters,
            temperature_model_parameters=self.temperature_model_parameters,
        )
        system = PVSystem(arrays=[array], inverter_parameters=self.inverter_parameters)
        mc = ModelChain(system, location, aoi_model="no_loss", spectral_model="no_loss")

        weather_data = pvlib.iotools.get_pvgis_tmy(row['latitude'], row['longitude'])[0]
        weather = weather_data[['ghi', 'dhi', 'dni', 'temp_air', 'wind_speed']]

        mc.run_model(weather)
        return {"result": {row['name']: mc.results.ac.sum()}}



### Définition du serveur MLFLOW et enregistrement du modèle

In [ ]:
mlflow.set_tracking_uri("http://model-platform.com/registry/pv/")

In [ ]:

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="custom_model",
        python_model=PVLibModel(),
        registered_model_name="pv_forecast_model"
    )
    mlflow.set_tag("mlflow.note.content", open("model_card.md", "r").read())

print("✅ Modèle loggué avec succès sure MLflow !")
